# NBA Single-Player Search
**Programmer:** Rahim Hashim<br>
**Date:** Sep 2021<br>
**Description:** Scrape basketball-reference.com by single players.<br>

Data captured includes —<br>
* Player Bios<br>
  * *Body Measurements* (height, weight, age...)
  * *Draft* (round, pick, year...)
  * *Hometown* 
  * *College Team*
* Season<br>
  * *Per Game* (pts/game, reb/game, assists/game...)
  * *Totals* (pts, reb, assists...)
  * *Per 36 Minutes* (pts, reb, assists...)
  * *Per 100 Possessions* (pts, reb, assists...)
  * *Advanced* (PER, OBPM, DBPM, VORP...)
  * *Play-by-Play* (PG%, SG%, And1, Blkd...)
  * *Shooting* (FG% by distance, Dunks, Corner 3s...)
* Game Logs <br>
  * Basic and Advanced stats 
  * Win-Loss
* Play-By-Play
  * Real time elapsed between shots, lineups

***

In [1]:
%reload_ext autoreload
import os
import sys
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict, defaultdict
pd.options.mode.chained_assignment = None  # default='warn'

ROOT = '/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/' #@param ['/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/']  

def add_helpers():
  '''
  add_helper mounts google drive and adds
  helper functions to the sys.path
  '''

  # if running on juypter/google colab, mount to google drive
  if 'google.colab' in str(get_ipython()): 
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    os.chdir(ROOT)

  helper_dir_path = ROOT + 'helper/'
  sys.path.append(helper_dir_path) # set to path of notebook

add_helpers()

Mounted at /content/drive


In [2]:
from player_scraper import single_player_scraper

kb_meta, kb_data, kb_gamelogs = single_player_scraper('Kobe Bryant')
kb_meta

Searching for name in: https://www.basketball-reference.com/players/b
  Joe Bryant (match score=0.857)
  Kobe Bryant (match score=1.0)
Best match: Kobe Bryant


,birthCountry,birthCity,birthState,shootingHand,highSchool,highSchoolCity,highSchoolState,highSchoolCountry,draftTeam,draftRound,draftRoundPick,draftOverallPick
0,United States of America,Philadelphia,Pennsylvania,Right,Lower Merion,Ardmore,Pennsylvania,United States of America,Charlotte Hornets,1,13,13


In [3]:
#@title Table Select { run: "auto" }

#@markdown Per Game | Totals | Advanced | Per Minute | Per Possession | Adjusted Shooting | Play-By-Play | Shooting | All-Star | Salaries
table_type = "per_game" #@param ["all", "per_game", "totals", "advanced", "per_minute", "per_poss", "adjooting", "pbp", "shooting", "all_star", "all_salaries"] {allow-input: true}

#@markdown Season or Playoff Stats
season_playoffs = "both" #@param ["both", "season", "playoffs"]

def table_select(df, table_type, season_playoffs):
  df_filtered = df.copy(deep=True)
  if table_type != 'all':
    df_filtered = df[df['data_type'] == table_type]
    df_filtered = df_filtered.dropna(how='all', axis='columns')
  if season_playoffs != 'both':
    df_filtered = df_filtered[df_filtered['season_playoffs'] == season_playoffs]
  return df_filtered

df_table = table_select(kb_data, table_type, season_playoffs)
df_table

,data_type,season_playoffs,player_name,season,age,team_id,lg_id,pos,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,fg2_per_g,fg2a_per_g,fg2_pct,efg_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g
0,per_game,season,Kobe Bryant,1996-97,18,LAL,NBA,SF,71,6,15.5,2.5,5.9,0.417,0.7,1.9,0.375,1.8,4.0,0.437,0.477,1.9,2.3,0.819,0.7,1.2,1.9,1.3,0.7,0.3,1.6,1.4,7.6
1,per_game,season,Kobe Bryant,1997-98,19,LAL,NBA,SF,79,1,26.0,4.9,11.6,0.428,0.9,2.8,0.341,4.0,8.8,0.456,0.469,4.6,5.8,0.794,1.0,2.1,3.1,2.5,0.9,0.5,2.0,2.3,15.4
2,per_game,season,Kobe Bryant,1998-99,20,LAL,NBA,SG,50,50,37.9,7.2,15.6,0.465,0.5,2.0,0.267,6.7,13.6,0.494,0.482,4.9,5.8,0.839,1.1,4.2,5.3,3.8,1.4,1.0,3.1,3.1,19.9
3,per_game,season,Kobe Bryant,1999-00,21,LAL,NBA,SG,66,62,38.2,8.4,17.9,0.468,0.7,2.2,0.319,7.7,15.7,0.489,0.488,5.0,6.1,0.821,1.6,4.7,6.3,4.9,1.6,0.9,2.8,3.3,22.5
4,per_game,season,Kobe Bryant,2000-01,22,LAL,NBA,SG,68,68,40.9,10.3,22.2,0.464,0.9,2.9,0.305,9.4,19.3,0.489,0.484,7.0,8.2,0.853,1.5,4.3,5.9,5.0,1.7,0.6,3.2,3.3,28.5
5,per_game,season,Kobe Bryant,2001-02,23,LAL,NBA,SG,80,80,38.3,9.4,20.0,0.469,0.4,1.7,0.25,9.0,18.3,0.489,0.479,6.1,7.4,0.829,1.4,4.1,5.5,5.5,1.5,0.4,2.8,2.9,25.2
6,per_game,season,Kobe Bryant,2002-03,24,LAL,NBA,SG,82,82,41.5,10.6,23.5,0.451,1.5,4.0,0.383,9.1,19.5,0.465,0.483,7.3,8.7,0.843,1.3,5.6,6.9,5.9,2.2,0.8,3.5,2.7,30.0
7,per_game,season,Kobe Bryant,2003-04,25,LAL,NBA,SG,65,64,37.6,7.9,18.1,0.438,1.1,3.3,0.327,6.8,14.8,0.463,0.468,7.0,8.2,0.852,1.6,3.9,5.5,5.1,1.7,0.4,2.6,2.7,24.0
8,per_game,season,Kobe Bryant,2004-05,26,LAL,NBA,SG,66,66,40.7,8.7,20.1,0.433,2.0,5.9,0.339,6.7,14.2,0.472,0.482,8.2,10.1,0.816,1.4,4.5,5.9,6.0,1.3,0.8,4.1,2.6,27.6
9,per_game,season,Kobe Bryant,2005-06,27,LAL,NBA,SG,80,80,41.0,12.2,27.2,0.45,2.3,6.5,0.347,10.0,20.7,0.482,0.491,8.7,10.2,0.85,0.9,4.4,5.3,4.5,1.8,0.4,3.1,2.9,35.4


In [4]:
type(df_table['pts_per_g'].iloc[5])

numpy.float64

In [5]:
kb_gamelogs

,player_name,season,season_playoffs,rk,g,date,age,tm,loc,opp,win,gs,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,gmsc,plus_minus,note
0,Kobe Bryant,1996-97,season,1,,1996-11-01,18-070,LAL,,PHO,W (+14),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Did Not Play
1,Kobe Bryant,1996-97,season,2,1,1996-11-03,18-072,LAL,,MIN,W (+6),0,6:22,0,1,0,0,0,,0,0,,0,1,1,0,0,1,1,1,0,-1.1,-8,
2,Kobe Bryant,1996-97,season,3,2,1996-11-05,18-074,LAL,@,NYK,W (+6),0,3:16,0,1,0,0,0,,1,2,0.5,0,0,0,0,0,0,1,0,1,-1.1,-8,
3,Kobe Bryant,1996-97,season,4,3,1996-11-06,18-075,LAL,@,CHH,L (-10),0,6:49,2,3,0.667,1,2,0.5,0,0,,0,0,0,0,0,0,3,0,5,0.7,0,
4,Kobe Bryant,1996-97,season,5,4,1996-11-08,18-077,LAL,@,TOR,L (-1),0,17:15,3,8,0.375,2,3,0.667,2,4,0.5,0,3,3,0,0,1,0,3,10,5.2,2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Kobe Bryant,2015-16,season,78,62,2016-04-06,37-227,LAL,,LAC,L (-10),1,27:46,6,19,0.316,1,6,0.167,4,4,1,0,3,3,1,1,0,1,1,17,7.3,-15,
78,Kobe Bryant,2015-16,season,79,63,2016-04-08,37-229,LAL,@,NOP,L (-8),1,22:13,4,15,0.267,3,9,0.333,3,3,1,0,3,3,4,1,0,2,0,14,7.8,-16,
79,Kobe Bryant,2015-16,season,80,64,2016-04-10,37-231,LAL,@,HOU,L (-20),1,27:26,10,22,0.455,4,11,0.364,11,11,1,0,2,2,1,2,0,4,1,35,22.5,-10,
80,Kobe Bryant,2015-16,season,81,65,2016-04-11,37-232,LAL,@,OKC,L (-33),1,18:57,4,12,0.333,3,9,0.333,2,2,1,0,1,1,0,1,0,2,2,13,4.7,-24,


In [6]:
type(kb_gamelogs['gmsc'].iloc[5])

numpy.float64

[Play-By-Play Stuff](https://jman4190.medium.com/how-to-accessing-live-nba-play-by-play-data-f24e02b0a976)

In [7]:
kb_gamelogs

,player_name,season,season_playoffs,rk,g,date,age,tm,loc,opp,win,gs,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,gmsc,plus_minus,note
0,Kobe Bryant,1996-97,season,1,,1996-11-01,18-070,LAL,,PHO,W (+14),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Did Not Play
1,Kobe Bryant,1996-97,season,2,1,1996-11-03,18-072,LAL,,MIN,W (+6),0,6:22,0,1,0,0,0,,0,0,,0,1,1,0,0,1,1,1,0,-1.1,-8,
2,Kobe Bryant,1996-97,season,3,2,1996-11-05,18-074,LAL,@,NYK,W (+6),0,3:16,0,1,0,0,0,,1,2,0.5,0,0,0,0,0,0,1,0,1,-1.1,-8,
3,Kobe Bryant,1996-97,season,4,3,1996-11-06,18-075,LAL,@,CHH,L (-10),0,6:49,2,3,0.667,1,2,0.5,0,0,,0,0,0,0,0,0,3,0,5,0.7,0,
4,Kobe Bryant,1996-97,season,5,4,1996-11-08,18-077,LAL,@,TOR,L (-1),0,17:15,3,8,0.375,2,3,0.667,2,4,0.5,0,3,3,0,0,1,0,3,10,5.2,2,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Kobe Bryant,2015-16,season,78,62,2016-04-06,37-227,LAL,,LAC,L (-10),1,27:46,6,19,0.316,1,6,0.167,4,4,1,0,3,3,1,1,0,1,1,17,7.3,-15,
78,Kobe Bryant,2015-16,season,79,63,2016-04-08,37-229,LAL,@,NOP,L (-8),1,22:13,4,15,0.267,3,9,0.333,3,3,1,0,3,3,4,1,0,2,0,14,7.8,-16,
79,Kobe Bryant,2015-16,season,80,64,2016-04-10,37-231,LAL,@,HOU,L (-20),1,27:26,10,22,0.455,4,11,0.364,11,11,1,0,2,2,1,2,0,4,1,35,22.5,-10,
80,Kobe Bryant,2015-16,season,81,65,2016-04-11,37-232,LAL,@,OKC,L (-33),1,18:57,4,12,0.333,3,9,0.333,2,2,1,0,1,1,0,1,0,2,2,13,4.7,-24,
